<a href="https://colab.research.google.com/github/rastin-py/QA-and-RAG-on-Guilan-Food-Recipes/blob/main/QA_and_RAG_on_Guilan_Food_Recipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install libraries

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo
!pip install --no-deps xformers trl peft accelerate bitsandbytes evaluate rouge_score python-docx rank_bm25 lancedb
!pip install git+https://github.com/PanQiWei/AutoGPTQ.git@v0.4.2
!pip install overrides deprecation lance_namespace datasets==3.6.0


import re
import numpy as np

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
my_model_path = "/content/drive/MyDrive/Code/llama_3_1b_model"
embedding_model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
load_model_from_file = True

# Step 2: Load Dataset, Model and Tokenizer

In [ ]:
from datasets import load_dataset
from datasets import load_from_disk

# dataset = load_dataset("Gholamreza/pquad")
# dataset.save_to_disk("/content/drive/MyDrive/Code/pquad_dataset")
dataset = load_from_disk("/content/drive/MyDrive/Code/pquad_dataset")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 63994
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 7976
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 8002
    })
})


In [ ]:
from unsloth import FastLanguageModel
import torch


original_model, original_tokenizer = FastLanguageModel.from_pretrained(
    model_name = embedding_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


if load_model_from_file:
  llama_3_1b_model, llama_3_1b_tokenizer = FastLanguageModel.from_pretrained(
      model_name = my_model_path,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )

else:
  llama_3_1b_model, llama_3_1b_tokenizer = FastLanguageModel.from_pretrained(
      model_name = embedding_model_name,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

==((====))==  Unsloth 2025.9.4: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.9.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# Step 3: Preprocess and Prepare Dataset

In [ ]:
chat_prompt = """
### Instruction:
{}

### Context:
{}

### Question:
{}

### Answer:
{}"""

In [ ]:
import random

EOS_TOKEN = llama_3_1b_tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples, validating, all_contexts=None):
    """
    all_contexts: list of all possible contexts (dataset-wide),
                  used to sample irrelevant ones.
    """
    instruction = "Answer the question based on the provided context."
    references  = examples["context"]
    inputs      = examples["question"]
    outputs     = examples["answers"]

    texts = []
    questions = []
    answers = []
    contexts = []

    for input, reference, output in zip(inputs, references, outputs):
        chosen = random.sample(references, 2) + [reference] # Decided to mix in some irrelevant contexts with the correct answer to increase model's robustness

        random.shuffle(chosen)
        merged_references = "\n".join(chosen)

        # EOS for answer
        if len(output['text']) > 0:
            real_ans = output['text'][0]
        else:
            real_ans = None

        # Build final text
        if validating:
            text = chat_prompt.format(instruction, merged_references, input, "")
        else:
            text = chat_prompt.format(instruction, merged_references, input, real_ans)
            text = text + EOS_TOKEN

        texts.append(text)
        questions.append(input)
        contexts.append(merged_references)
        answers.append(real_ans)

    return {
        "text": texts,
        "context": contexts,
        "question": questions,
        "answer": answers
    }


In [ ]:
train_dataset = dataset["train"].map(
    formatting_prompts_func,
    batched=True,
    fn_kwargs={"validating": False, "all_contexts": dataset["train"]["context"]},
)

val_dataset = dataset["validation"].map(
    formatting_prompts_func,
    batched=True,
    fn_kwargs={"validating": True, "all_contexts":  dataset["validation"]["context"]},
)

test_dataset = dataset["test"].map(
    formatting_prompts_func,
    batched=True,
    fn_kwargs={"validating": True, "all_contexts":  dataset["test"]["context"]},
)


Map:   0%|          | 0/63994 [00:00<?, ? examples/s]

Map:   0%|          | 0/7976 [00:00<?, ? examples/s]

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

In [ ]:
# Drop unnecessary columns and rows with None values
train_dataset = train_dataset.remove_columns(['id', 'title', 'answers']).filter(lambda x: x['answer'] is not None)
val_dataset = val_dataset.remove_columns(['id', 'title', 'answers']).filter(lambda x: x['answer'] is not None)
test_dataset = test_dataset.remove_columns(['id', 'title', 'answers']).filter(lambda x: x['answer'] is not None)

Filter:   0%|          | 0/63994 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7976 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8002 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.shuffle(seed=42).select(range(2048))
val_dataset   = val_dataset.shuffle(seed=42).select(range(320))
test_dataset  = test_dataset.shuffle(seed=42).select(range(160))

In [ ]:
print("train data looks like: \n\n")
print("question is: ", train_dataset['question'][0], "\ncontext is:" ,train_dataset['context'][0], "\nanswer is:" ,train_dataset['answer'][0])
print('------')
print("model train data will be: ", train_dataset['text'][0])

train data looks like: 


question is:  اصفهان تا اوایل سده چهارم هجری تحت سلطه چه کسانی بوده‌است؟ 
context is: اصفهان تا اوایل سده چهارم هجری تخت سلطه خلفای عباسی بوده تا آنکه «مرداویج زیاری» از حکمرانان آل زیار در سال ۳۱۶ هـ. ق آنرا فتح می‌نماید. وی در راستای زنده کردن آیین‌های ایران باستان، در هنگام جشن سده (دهم بهمن ماه) آتش بازی و جشن بزرگی را در زمینهای میان زاینده رود و هزار دره که اراضی تخت فولاد و لسان الارض را در میان می‌گیرد بر پا می‌نماید. بر طبق این مطلب به گمان قوی تخت فولاد در آن زمان زمین لم بزرعی بیش نبوده‌است. از قرن اول تا سوم هجری آثاری در تخت فولاد به دست نیامده است، اما از قرن چهارم به بعد شواهدی بر تاریخ تخت فولاد موجود است. همچنین سنگ قبوری نیز متعلق به قرون چهارم و پنجم هجری در این منطقه توسط سازمان میراث فرهنگی یافت شده‌است.
نائین در استان اصفهان و مرکز شهرستان نائین قرار دارد. این شهر در منطقه دشت کویر قرار گرفته‌است و آب و هوایی خشک دارد. نایین در فاصله ۶۵ کیلومتری شهرستان ورزنه واقع شده و فاصله آن تا تالاب گاوخونی ۹۰ کیلومتر است.
تیپ و شکل آب‌ان

In [ ]:
print("validation data looks like: \n\n")
print("question is: ", val_dataset['question'][0], "\ncontext is:" ,val_dataset['context'][0], "\nanswer is:" ,val_dataset['answer'][0])
print('------')
print("model validation data will be: ", val_dataset['text'][0])

validation data looks like: 


question is:  طبق قانون مصوب در هجدهم اردیبهشت ۱۳۱۸ به پس‌اندازهای بیش از دو هزار تومان چقدر سود تعلق می‌گرفت؟ 
context is: تا سال ۱۳۱۴ سرمایه بانک ملی به سی میلیون تومان رسید. در سال ۱۳۱۶ رضا شاه بخشی از جواهرات سلطنتی را برای افزایش سرمایه بانک ملی به این بانک بخشید. در ۲۲ امرداد ۱۳۱۷ اساسنامه تازه‌ای برای بانک ملی به تصویب مجلس شورای ملی رسید که چاپ اسکناس را در انحصار بانک ملی قرار می‌داد. این اساسنامه همچنین مقرر می‌داشت که مدیر کل و قائم‌مقام و معاونان بانک و اعضای هفت‌گانه شورای عالی بانک باید ایرانی باشند. این شورا عهده‌دار صدور اجازه چاپ اسکناس و به جریان انداختن یا از جریان خارج کردن آن، تعیین نرخ رسمی تنزیل (بهره) به پیشنهاد مدیر کل و تعیین بهاء خرید و فروش ارز بود.
در سال ۱۳۱۸ با تشکیل «صندوق پس‌انداز ملی» تحت اداره بانک ملی ایران، حساب‌های پس‌انداز شکل گرفت که به سپرده‌گذاران سود بانکی می‌داد. بنابر قانون تأسیس این صندوق که در هجدهم اردیبهشت ۱۳۱۸ به تصویب رسید، به پس‌اندازهای کمتر از پانصد تومان، چهار درصد و پانصد تا دو هزار تومان، سه درصد

#Step 4: Configure the Model

In [ ]:
llama_3_1b_model = FastLanguageModel.get_peft_model(
    llama_3_1b_model,
    r = 20,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 18,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
import torch

trainer = SFTTrainer(
    model=llama_3_1b_model,
    tokenizer=llama_3_1b_tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=10,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=32,
        eval_steps=32,
        eval_strategy="steps",
        save_strategy="steps",
        save_total_limit=2,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/drive/MyDrive/Code/",
        report_to="none",
        load_best_model_at_end=True,
        save_steps=64,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    ),
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=5)  # stop if no improvement in 5 evals
    ],
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2048 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/320 [00:00<?, ? examples/s]

#Step 5: Start Training

In [ ]:
if not load_model_from_file:
  import wandb
  wandb.init(mode="disabled")
  trainer_stats = trainer.train()
  # trainer_stats = trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Code/checkpoint-180")

  trainer.model.save_pretrained(my_model_path)
  trainer.tokenizer.save_pretrained(my_model_path)
else:
  print('Llama Model was loaded from file.')

Llama Model was loaded from file.


#Step 6: Inference and Evaluation

In [ ]:
def generate_llm_answers(llm_model, llm_tokenizer, texts):
  FastLanguageModel.for_inference(llm_model)
  inputs = llm_tokenizer(
      texts, return_tensors="pt"
  ).to("cuda")
  with torch.no_grad():
    output = llm_model.generate(**inputs, max_new_tokens=128,temperature=0.2, use_cache=True)
  full_conversation = llm_tokenizer.batch_decode(output)  # Decode the output
  # Extracting the response part
  response = full_conversation[0].split("### Answer:")[-1].strip()  # Get text after "### Response:"
  response = response.split(EOS_TOKEN)[0].strip()  # Remove the end token if present
  return full_conversation[0], response



In [ ]:
sample_input_index = 20
full_conversations, answers_only = generate_llm_answers(llama_3_1b_model, llama_3_1b_tokenizer, val_dataset['text'][sample_input_index])

print('question:\n', val_dataset['question'][sample_input_index])
print('context:\n', val_dataset['context'][sample_input_index])
print('real answer:\n', val_dataset['answer'][sample_input_index])
print('generated answer:\n', answers_only)
print('--------')
print('entire conversation:\n', full_conversations)

question:
 دولت عثمانی در کدام دریا قدرت مطلق دریایی بود؟
context:
 امروزه این فدراسیون حدود ۲۱۷ کشور عضو دارد (سال ۲۰۰۷) و بیش از ۲۵۰ میلیون نفر از مردم جهان والیبال بازی می‌کنند. اولین رئیس فدراسیون جهانی والیبال پل لیبود تا سال ۱۹۸۴ (یعنی ۳۷ سال) ریاست را برعهده داشت. پس از تأسیس فدراسیون جهانی والیبال، کمیته‌های مختلفی در داخل آن به وجود آمد و برنامه مسابقات رسمی جهانی تنظیم و آغاز شد. در سال ۱۹۴۹ اولین دوره مسابقات جهانی والیبال برای مردان در پراگ و در سال ۱۹۵۲ دومین دوره مسابقات جهانی مردان و اولین دوره مسابقات جهانی زنان در مسکو برگزار شد. برنامه این مسابقات به‌طور منظم هر چهار سال یک بار تاکنون در کشورهای مختلف انجام شده‌است. در قهرمانی جهان ۲۰۱۴ لهستان نیز بعد از جام جهانی فوتبال دومین تورنمنت پربازدید ورزش جهان قلمداد شد.
روزی که سلطان سلیمان قانونی درگذشت، امپراتوری عثمانی نسبت به روز برتخت نشینی او ۲٬۲۷۳٬۷۲۰ کیلومتر مربع وسیع‌تر بود و در سه قاره مناطقی را تحت فرمان خود داشت. در کنار این مسئله، دولت عثمانی در دریای مدیترانه نیز قدرت مطلق دریایی بود و به یکی از بازیگران

In [ ]:
import evaluate
# Load metrics
bleu = evaluate.load("bleu")
import re
from collections import Counter

def tokenize(text):
    """Simple whitespace + punctuation tokenizer"""
    text = re.sub(r"[^آ-یA-Za-z0-9]+", " ", text)  # Persian + English
    return text.strip().split()

def ngrams(tokens, n):
    return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def rouge_n(prediction, reference, n=1):
    pred_ngrams = Counter(ngrams(prediction, n))
    ref_ngrams  = Counter(ngrams(reference, n))
    overlap = sum((pred_ngrams & ref_ngrams).values())

    recall = overlap / max(1, sum(ref_ngrams.values()))
    precision = overlap / max(1, sum(pred_ngrams.values()))
    f1 = 2*recall*precision/(recall+precision) if (recall+precision)>0 else 0.0
    return {"recall": recall, "precision": precision, "f1": f1}

def lcs(X, Y):
    m, n = len(X), len(Y)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if X[i] == Y[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    return dp[m][n]

def rouge_l(prediction, reference):
    lcs_len = lcs(prediction, reference)
    recall = lcs_len / max(1, len(reference))
    precision = lcs_len / max(1, len(prediction))
    f1 = 2*recall*precision/(recall+precision) if (recall+precision)>0 else 0.0
    return {"recall": recall, "precision": precision, "f1": f1}

def rouge_lsum(pred_sents, ref_sents):
    recalls, precisions, f1s = [], [], []
    for ref in ref_sents:
        best = {"recall":0, "precision":0, "f1":0}
        for pred in pred_sents:
            score = rouge_l(pred, ref)
            if score["f1"] > best["f1"]:
                best = score
        recalls.append(best["recall"])
        precisions.append(best["precision"])
        f1s.append(best["f1"])
    return {
        "recall": sum(recalls)/len(recalls) if recalls else 0.0,
        "precision": sum(precisions)/len(precisions) if precisions else 0.0,
        "f1": sum(f1s)/len(f1s) if f1s else 0.0
    }

def compute_rouge(predictions, references):
    assert len(predictions) == len(references), "Mismatched lengths!"
    scores = {"rouge1": [], "rouge2": [], "rougeL": [], "rougeLsum": []}

    for pred, ref in zip(predictions, references):
        pred_tokens = tokenize(pred)
        ref_tokens  = tokenize(ref)

        pred_sents = [tokenize(s) for s in pred.split(" . ")]
        ref_sents  = [tokenize(s) for s in ref.split(" . ")]

        scores["rouge1"].append(rouge_n(pred_tokens, ref_tokens, n=1))
        scores["rouge2"].append(rouge_n(pred_tokens, ref_tokens, n=2))
        scores["rougeL"].append(rouge_l(pred_tokens, ref_tokens))
        scores["rougeLsum"].append(rouge_lsum(pred_sents, ref_sents))

    # Average across all examples
    final_scores = {}
    for key, vals in scores.items():
        final_scores[key] = {
            "recall": sum(v["recall"] for v in vals)/len(vals),
            "precision": sum(v["precision"] for v in vals)/len(vals),
            "f1": sum(v["f1"] for v in vals)/len(vals),
        }
    return final_scores


In [ ]:
preds, refs  = [], []

for tst in test_dataset['text']:
  full_conversations, generated_answer = generate_llm_answers(llama_3_1b_model, llama_3_1b_tokenizer, tst)
  preds.append(generated_answer if not None else '')


refs = test_dataset['answer']

# Compute metrics
rouge_result = compute_rouge(predictions=preds, references=refs)
bleu_result = bleu.compute(predictions=preds, references=[[r] for r in refs])

# Exact Match Accuracy (manual)
exact_matches = [int(p.strip().lower() == r.strip().lower()) for p, r in zip(preds, refs)]
exact_match_acc = sum(exact_matches) / len(exact_matches)

In [ ]:
print("🔹 Evaluation Results:")
print("ROUGE:", rouge_result)
print("BLEU:", bleu_result)
print("Exact Match Accuracy:", exact_match_acc)

🔹 Evaluation Results:
ROUGE: {'rouge1': {'recall': 0.5501499961265586, 'precision': 0.5634666996968349, 'f1': 0.5174153699519874}, 'rouge2': {'recall': 0.3722567722335336, 'precision': 0.3827368241714022, 'f1': 0.3462214903489321}, 'rougeL': {'recall': 0.5496291627932253, 'precision': 0.5625051612352964, 'f1': 0.5167396942763117}, 'rougeLsum': {'recall': 0.5496291627932253, 'precision': 0.5625051612352964, 'f1': 0.5167396942763117}}
BLEU: {'bleu': 0.4366114355975146, 'precisions': [0.5358490566037736, 0.47086614173228347, 0.43545279383429675, 0.40857787810383744], 'brevity_penalty': 0.9485410718192575, 'length_ratio': 0.9498207885304659, 'translation_length': 795, 'reference_length': 837}
Exact Match Accuracy: 0.3375


In [ ]:
print("\n🔹 Sample Examples:")
for i in range(5): # Show 5 examples
    print(f"\n--- Example {i+1} ---")
    print("Question:", test_dataset['question'][i])
    print("Context:", test_dataset['context'][i])
    print("Real Answer:", test_dataset['answer'][i])
    print("Generated Answer:", preds[i])


🔹 Sample Examples:

--- Example 1 ---
Question: مقدار نیروی جاذبه اضافی نسبت به هر اتم اکسیژن موجود در مولکول‌های مجاور چقدر است؟
Context: امپراتور پروباس (Probus) برای حمله به قلمرو ساسانیان طرح‌ریزی کرد، اما خودش مرد و کاروس (Carus) جنگ را شروع و به بین‌النهرین حمله کرد و تیسفون را محاصره کرد. در این زمان بهرام دوم در شرق بود. اما کاروس در بین‌النهرین درگذشت و امپراتور بعدی، دیوکلتیان، می‌بایست مشغول به حل مشکلات داخلی می‌شد و با بهرام دوم صلح کرد، که بر طبق آن ارمنستان بین دو کشور تقسیم شد و بخش غربی در دست تیرداد ماند و نرسه بخش بزرگ‌تر را گرفت (که از آن پس ارمنستان ایران نام گرفت). در ۲۹۳ بهرام دوم درگذشت و بهرام سوم که شاه سیستان بود بر تخت نشست، اما نرسه، برادر بهرام دوم، که در شرق بود، به میان‌رودان آمد و مورد استقبال جمعی از اشراف قرار گرفت و در نهایت شاه شد.
آب (H۲O) اکسید هیدروژن و آشناترین ترکیب اکسیژن است. در یک مولکول آب، هر یک از دو اتم هیدروژن موجود از طریق یک پیوند کووالانسی و مستقل با اتم اکسیژن مرکزی پیوند می‌دهند اما آنها علاوه‌بر این اتصال، دارای یک نیروی ج

# Step 7: RAG, Load Document and Chunking

In [ ]:
import docx

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)

file_path = "/content/drive/MyDrive/Code/Guilan-Food.docx"
guilan_food_text = read_docx(file_path)

print(guilan_food_text[:500], '...')

راهنمای آشپزی غذاهای استان گیلان

استان گیلان به خاطر تنوع و طعم بی‌نظیر غذاهایش شهرت دارد. این منطقه با داشتن آب و هوای معتدل و دسترسی به دریای خزر و جنگل‌های سرسبز، مواد اولیه تازه و متنوعی را در اختیار دارد که در آشپزی محلی به خوبی از آن‌ها استفاده می‌شود. در ادامه چند مورد از غذاهای معروف و محبوب استان گیلان را آمده است.
باقلا قاتوق
باقلا قاتوق یکی از محبوب‌ترین و خوشمزه‌ترین خورشت‌های استان گیلان است که طعم و عطر بی‌نظیری دارد. این خورشت ساده اما بسیار مقوی و لذیذ است و معمولاً با کته (برنج ...


In [ ]:
def chunk_text_by_word(text, chunk_size=120, overlap=40):
  words = text.split(' ')
  words = [w.strip() for w in words if w.strip() not in ("")]
  chunks = []
  start = 0
  while start < len(words):
    end = start + chunk_size
    chunk = " ".join(words[start:end])
    chunks.append(chunk)
    start = end - overlap
  return chunks


def chunk_text_by_sentence(text, chunk_size=9, overlap=3):
  sentences = re.split(r'[.\n]', text)
  sentences = [s.strip() for s in sentences if s.strip() not in ("")]
  chunks = []
  start = 0
  while start < len(sentences):
    end = start + chunk_size
    chunk = " ".join(sentences[start:end])
    chunks.append(chunk)
    start = end - overlap
  return chunks

def chunk_text_by_paragraph(text, chunk_size=3, overlap=1):
  paragraphs = re.split(r'[\n]', text)
  paragraphs = [p.strip() for p in paragraphs if p.strip() not in ("")]
  chunks = []
  start = 0
  while start < len(paragraphs):
    end = start + chunk_size
    chunk = " ".join(paragraphs[start:end])
    chunks.append(chunk)
    start = end - overlap
  return chunks


# chuknings
word_based_chunk = chunk_text_by_word(guilan_food_text)
sentence_based_chunk = chunk_text_by_sentence(guilan_food_text)
paragraph_based_chunk = chunk_text_by_paragraph(guilan_food_text)
meta_based_chunk = word_based_chunk + sentence_based_chunk + paragraph_based_chunk


In [ ]:
print(len(word_based_chunk), len(sentence_based_chunk), len(paragraph_based_chunk), len(meta_based_chunk))

53 63 100 216


# Step 8: Set up Embedding Database

In [ ]:
from lancedb import connect
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import numpy as np
import lancedb

def create_embedding_databases(embedding_model_name, word_based_chunk, sentence_based_chunk, paragraph_based_chunk, meta_based_chunk):
    embedding_model = SentenceTransformer(embedding_model_name)

    # Meta chunk = combine all granularities
    meta_based_chunk = word_based_chunk + sentence_based_chunk + paragraph_based_chunk

    # Encode embeddings
    word_chunk_embeddings = embedding_model.encode(word_based_chunk, normalize_embeddings=True)
    sentence_chunk_embeddings = embedding_model.encode(sentence_based_chunk, normalize_embeddings=True)
    paragraph_chunk_embeddings = embedding_model.encode(paragraph_based_chunk, normalize_embeddings=True)
    meta_chunk_embeddings = embedding_model.encode(meta_based_chunk, normalize_embeddings=True)

    # Create a LanceDB database
    db = lancedb.connect("/tmp/lancedb")

    # Create tables for word, sentence, paragraph, and meta chunks
    word_chunk_table = db.create_table(
        f"{embedding_model_name.replace('/', '_')}_word_chunks",
        data=[
            {"vector": embedding.tolist(), "text": chunk}
            for embedding, chunk in zip(word_chunk_embeddings, word_based_chunk)
        ],
        mode="overwrite"
    )

    sentence_chunk_table = db.create_table(
        f"{embedding_model_name.replace('/', '_')}_sentence_chunks",
        data=[
            {"vector": embedding.tolist(), "text": chunk}
            for embedding, chunk in zip(sentence_chunk_embeddings, sentence_based_chunk)
        ],
        mode="overwrite"
    )

    paragraph_chunk_table = db.create_table(
        f"{embedding_model_name.replace('/', '_')}_paragraph_chunks",
        data=[
            {"vector": embedding.tolist(), "text": chunk}
            for embedding, chunk in zip(paragraph_chunk_embeddings, paragraph_based_chunk)
        ],
        mode="overwrite"
    )

    meta_chunk_table = db.create_table(
        f"{embedding_model_name.replace('/', '_')}_meta_chunks",
        data=[
            {"vector": embedding.tolist(), "text": chunk}
            for embedding, chunk in zip(meta_chunk_embeddings, meta_based_chunk)
        ],
        mode="overwrite"
    )

    # BM25 indices
    bm25_word = BM25Okapi([doc.split() for doc in word_based_chunk])
    bm25_sentence = BM25Okapi([doc.split() for doc in sentence_based_chunk])
    bm25_paragraph = BM25Okapi([doc.split() for doc in paragraph_based_chunk])
    bm25_meta = BM25Okapi([doc.split() for doc in meta_based_chunk])

    return (
        embedding_model,
        word_chunk_table,
        sentence_chunk_table,
        paragraph_chunk_table,
        meta_chunk_table,
        bm25_word,
        bm25_sentence,
        bm25_paragraph,
        bm25_meta,
    )

# ---- All Embedding Models ----
all_embeddings = {}
embedding_models = [
    "paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/distiluse-base-multilingual-cased-v2",
    "intfloat/multilingual-e5-base"
]

for model_name in embedding_models:
    embedding_data = create_embedding_databases(
        model_name, word_based_chunk, sentence_based_chunk, paragraph_based_chunk, meta_based_chunk
    )

    all_embeddings[model_name] = {
        "model": embedding_data[0],
        "word_db": embedding_data[1],
        "sentence_db": embedding_data[2],
        "paragraph_db": embedding_data[3],
        "meta_db": embedding_data[4],
        "bm25_word": embedding_data[5],
        "bm25_sentence": embedding_data[6],
        "bm25_paragraph": embedding_data[7],
        "bm25_meta": embedding_data[8],
    }

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
def retrieve_contexts(queries, embedding_model, vector_db_table, bm25_index, docs, k, alpha):
    """
    Hybrid retrieval: BM25 + LanceDB for multiple queries
    Returns: list of lists of contexts for each query
    """
    all_contexts = []
    for query in queries:
        # ---- BM25 scores ----
        tokenized_query = query.split()
        bm25_scores = bm25_index.get_scores(tokenized_query)

        # ---- Embedding similarity scores ----
        query_embedding = embedding_model.encode([query], normalize_embeddings=True)[0]
        results = vector_db_table.search(query_embedding).limit(len(docs)).to_list()
        embed_scores = [r['_distance'] for r in results]
        embed_scores = 1 - np.array(embed_scores)  # distance → similarity

        # ---- Normalize ----
        bm25_norm = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + 1e-8)
        embed_norm = (embed_scores - np.min(embed_scores)) / (np.max(embed_scores) - np.min(embed_scores) + 1e-8)

        # ---- Fuse scores ----
        final_scores = alpha * bm25_norm + (1 - alpha) * embed_norm
        ranked_idx = np.argsort(final_scores)[::-1][:k]

        # Append top-k docs for this query
        all_contexts.append([docs[i] for i in ranked_idx])

    return all_contexts


In [ ]:
queries = ["طرز تهیه گمج کباب"]
embedding_model = all_embeddings["intfloat/multilingual-e5-base"]["model"]

results = retrieve_contexts(
    queries,
    embedding_model,
    all_embeddings["intfloat/multilingual-e5-base"]["meta_db"],
    all_embeddings["intfloat/multilingual-e5-base"]["bm25_meta"],
    meta_based_chunk,
    k=5,
    alpha=0.5
)

for r in results[0]:
  print(r)
  print("----------")


یا نان سرو می‌شود.
9.گمج کباب گمج کباب ، یکی از غذاهای بسیار خوشمزه گیلانی است که نوعی از پخت کباب در ظروف گلی به نام گمج می باشد. همچنین امروزه این غذا به عنوان یکی از اصلی ترین غذاهای رستوران های گیلان تبدیل شده است. مواد لازم برای تهیه گمج کباب:
گوشت بدون چربی و قورمه ای خرد شده :250 گرم
پیاز نگینی شده :یک عدد
پودر گردو :60 گرم
رب گوجه فرنگی :2 قاشق غذا خوری
آب انار ترش :یک فنجان
رب انار:2 قاشق غذاخوری
سبزی شامل تره، جعفری و گشنیز خرد شده :یک فنجان
آب :نصف لیوان
روغن :به میزان لازم
نمک :به میزان لازم
طرز تهیه گمج کباب : قابلمه ای را روی حرارت قرار دهید و مقداری روغن داخلش بریزید تا داغ شود. سپس پیاز های خرد
----------
طرز تهیه کال کباب : تمام مواد را با هم مخلوط کرده و در تابه با روغن داغ بریزید مانند کوکوی معمولی، هر دو طرف را سرخ کنید اشپل کوکو طعمی متفاوت و مغذی دارد و معمولاً با برنج یا نان سرو می‌شود 9 گمج کباب گمج کباب ، یکی از غذاهای بسیار خوشمزه گیلانی است که نوعی از پخت کباب در ظروف گلی به نام گمج می باشد همچنین امروزه این غذا به عنوان یکی از اصلی ترین غذاهای رستوران های گی

# Step 9: Choosing best RAG method

In [55]:
test_data_for_rag = {
    'questions': [
        "اشپل کوکو از چه چیزهایی درست می‌شود؟",
        "پنیر برشته به چی شناخته می شود؟",
        "پنیر برشته از غذاهای سنتی کجا است؟",
        "میرزا قاسمی معمولاً به عنوان چه چیزی سرو می‌شود؟",
        "استان گیلان به خاطر چه چیزی شهرت دارد؟",
        "اناربیج به چه غذایی شباهت‌هایی دارد؟",
        "برای تهیه رشته خشکار (خشکار (برای 4 نفر) به چند پیمانه شیر لازم است؟",
        "برای تهیه کال کباب چند عدد تخم مرغ لازم است؟",
        "مرحله پخت باقالاها حدودا چند دقیق زمان می‌برد؟",
        "طعم منحصر به فرد کباب ترش به دلیل چیست؟",
        "برای تهیه میرزا قاسمی برای 4 نفر به چند عدد بامدجان نیاز است؟",
        "واویشکا ریشه در چه کشوری دارد؟",
        "نام دیگر سیرابیج چیست؟",
        "برای خورش انار مسمه به چند قاشق رب انار نیاز است؟",
        "مواد لازم برای تهیه رشته خشکار (برای 4 نفر)",
        "مواد لازم برای تهیه کال کباب (برای 4 نفر)",
        "طرز تهیه خورشت چغرتمه؟",
        "طرز تهیه کولی غورابیج یا کوله غورابه به چه صورت است؟",
        "طرز تهیه خورش انار مسما :"
    ],
    'answers': [
        "تخم ماهی (اشپل)، تخم مرغ و سبزیجات",
        "پنیر وورووشته، پنیر وروشون",
        "شمال کشور",
        "نان تازه.",
        "تنوع و طعم بی‌نظیر غذاهایش",
        "فسنجان",
        "2 پیمانه",
        "4 عدد",
        "حدود 20 تا 30 دقیقه",
        "استفاده از مواد اولیه‌ای چون گردو، رب انار و سبزیجات معطر محلی",
        "3-4 عدد",
        "روسیه",
        "سیراویج",
        "۴ قاشق غذاخوری",
        "آرد برنج:   1 پیمانه\nشیر:  2 پیمانه\nشکر:  1 پیمانه\nگلاب:  1 قاشق غذاخوری\nپودر هل:  1/2 قاشق چایخوری\nپودر نارگیل : به مقدار لازم\nکره یا روغن مایع:  1 قاشق غذاخوری\nپسته یا بادام خرد شده (اختیاری) : به مقدار لازم\nآب:  1/2  پیمانه",
        "• بادمجان کبابی: 3-4 عدد\n• گردوی آسیاب‌شده: 100 گرم\n• رب انار ترش: 2 قاشق غذاخوری\n• سیر: 2 حبه\n• چوچاق (در صورت دسترسی): 1 قاشق غذاخوری\n• نمک: به مقدار لازم",
        "مرغ پخته شده را ریش‌ریش و یا خرد کنید و با کمی روغن تفت دهید. بعد رب گوجه را به آن اضافه کنید و هنگامی که رب رنگ باز کرد، پیاز را اضافه کنید و با حرارت کم رب و پیاز را تفت دهید.بعد آب مرغ را به همراه آبلیمو و ادویه ها به اندازه ای که روی مواد را بگیرد اضافه کنید و بگذارید تا مواد به مدت 5 دقیقه با هم پزند.بعد زعفران را هم اضافه کنید و بعد از 5 تا 10 دقیقه جوشیدن و به روغن افتادن خورش، می توانید تخم مرغها را اضافه کنید.اما قبلش باید تخم مرغ ها را در ظرفی جداگانه با چنگال هم بزنید تا یکدست شده باشند.وقتی تخم مرغ ها را روی خورش ریختید کمی هم بزنید تا به همه جای غذا برسد.درب ظرف را بگذارید تا تخم مرغ خودشان را بگیرند و بسته شوند.خورش چغرتمه آماده شده نباید خیلی آبکی باشد.",
        "این خورشت تا حدی به خورشت فسنجان شبیه است و برای تهیه آن به گردو نیاز داریم. گردو را خرد کنید. برای خورشت‌تان غلظت بهتری داشته باشد می‌توانید گردو را چرخ کنید یا در غذاساز بریزید. سپس گردوها را به همراه ۲ لیوان آب در قابلمه ریخته و روی حرارت قرار دهید. وقتی به جوش آمد زیر آن را کم کنید تا برای یک ساعت بپزد و روغن بیاندازد. سر و دم ماهی کولی را بگیرید، شکم آن را خالی کنید و ماهی را بشویید. اگر بتوانید پوست ماهی را جدا کنید، خورش غورابیج شما طعم بهتری خواهد داشت. ماهی را با نمک و فلفل مزه‌دار کرده و در کمی روغن سرخ کنید. ماهی کولی برای پخت زمان زیادی لازم ندارد، پس کنار اجاق بمانید تا نسوزد. سزیجات را در تفت دهید تا آب آنها گرفته شود. سپس آب غوره، مخلوط گردو و ماهی‌های سرخ ‌شده را به آنها اضافه کنید. شعله را کم کنید تا خورشت غورابیج به آرامی بپزد و مزه‌ها به خورد هم بروند.خورشت کولی غورابیج شما آماده است.",
        "در اولین مرحله از طرز تهیه انار مسما باید تکه‌های مرغ را با مقداری نمک، فلفل، زردچوبه و نصف مقدار زعفران دم‌کرده گفته شده در دستور مزه‌دار کنید. سپس کمی روغن در ماهیتابه بریزید و مرغ‌ها را در روغن سرخ کنید تا طلایی شوند. بعد از این که مرغ‌ها طلایی شدند آن‌ها را از روغن خارج کنید و کنار بگذارید. در این مرحله باید پیاز را به صورت نگینی و ریز سرخ کنید. سپس در روغن تفت دهید. سپس سیر رنده شده را به آن اضافه کنید و بعد از این که پیازها سبک شدند، نمک، فلفل و زردچوبه را اضافه کنید. در سومین مرحله از انار مسما گیلانی ترش و خوشمزه با مرغ باید رب گوجه را به ترکیب پیازها اضافه کنید و خوب در روغن تفت دهید تا بوی خامی رب گرفته شود. در این مرحله از طرز تهیه انار مسما گیلانی باید مرغ‌های سرخ شده را همراه با ۱ لیوان آب جوش به سایر مواد اضافه کنید. سپس درب قابلمه را روی آن قرار دهید و صبر کنید تا مرغ به آرامی بپزد. چند دقیقه بعد از پخت مرغ‌ها، باید رب انار، آب انار و رب انار را به خورش اضافه کنید. سپس دوباره درب قابلمه را روی آن قرار دهید تا خورش بپزد و جا بیفتد. در اواخر پخت باقی مانده زعفران دم‌کرده را به خورش اضافه کنید. سپس خورش انار مسما را در ظرف مورد نظر بریزید و همراه با برنج کته یا برنج چلو رستورانی میل کنید. برای تزیین این خورش می‌توانید از دانه انار استفاده کنید."
    ]
}


In [ ]:
def full_qa(query, embedding_model_name, chunking_type, llm_model, llm_tokenizer, k, alpha, max_new_tokens=256):
    embeddings_model = all_embeddings[embedding_model_name]['model']

    # Select correct DBs and chunks based on chunking type
    if chunking_type=='sentence':
        vector_db_table = all_embeddings[embedding_model_name]["sentence_db"]
        bm25_index = all_embeddings[embedding_model_name]["bm25_sentence"]
        docs = sentence_based_chunk
    elif chunking_type=='word':
        vector_db_table = all_embeddings[embedding_model_name]["word_db"]
        bm25_index = all_embeddings[embedding_model_name]["bm25_word"]
        docs = word_based_chunk
    elif chunking_type=='paragraph':
        vector_db_table = all_embeddings[embedding_model_name]["paragraph_db"]
        bm25_index = all_embeddings[embedding_model_name]["bm25_paragraph"]
        docs = paragraph_based_chunk
    elif chunking_type=='meta':
        vector_db_table = all_embeddings[embedding_model_name]["meta_db"]
        bm25_index = all_embeddings[embedding_model_name]["bm25_meta"]
        docs = meta_based_chunk
    else:
        raise Exception("incorrect chunking method")


    # Step 1: Retrieve contexts
    contexts = retrieve_contexts(
    [query],
    embeddings_model,
    vector_db_table,
    bm25_index,
    docs,
    k=k,
    alpha=alpha
    )
    contexts = contexts[0]
    context_text = "\n".join(contexts)
    prompt = chat_prompt.format("Answer the question based on the provided context.", context_text, query, "")

    # Step 4: Handle Hugging Face or API-based models
    if hasattr(llm_model, "generate"):  # Hugging Face LLaMA/Mistral/etc.
        _ , answers = generate_llm_answers(llama_3_1b_model, llama_3_1b_tokenizer, [prompt])
    else:
        raise ValueError("Unsupported model type: expected SentenceTransformer")

    return (answers, contexts)



In [ ]:
q = 'میرزا قاسمی غذای کدام استان است؟'

answer, contexts = full_qa(q, "sentence-transformers/distiluse-base-multilingual-cased-v2", 'paragraph',  llama_3_1b_model, llama_3_1b_tokenizer, k=5, alpha=0.8, max_new_tokens=256)
print(answer)
print('------')
print('contexts: ')
for c in contexts:
  print(c)
  print('------')

استان گیلان
------
contexts: 
میرزا قاسمی میرزا قاسمی یکی دیگر از غذاهای بسیار محبوب و خوشمزه استان گیلان است که به سادگی تهیه می‌شود و طعم دودی و لذیذی دارد. این غذا معمولاً به عنوان پیش‌غذا یا یک وعده غذایی سبک همراه با نان تازه سرو می‌شود. طعم اصلی میرزا قاسمی ترکیبی از بادمجان کبابی با عطر دودی، طعم ترش و شیرین گوجه فرنگی، تندی ملایم سیر و غنای تخم مرغ است. بافت آن نرم و کمی له شده است و به خوبی با نان مخلوط می‌شود. مواد لازم برای تهیه میرزا قاسمی (برای 4 نفر):
------
مواد لازم برای تهیه میرزا قاسمی (برای 4 نفر): بادمجان متوسط:  3-4  عدد گوجه فرنگی متوسط:  2-3 عدد (رسیده و آبدار)
------
راهنمای آشپزی غذاهای استان گیلان استان گیلان به خاطر تنوع و طعم بی‌نظیر غذاهایش شهرت دارد. این منطقه با داشتن آب و هوای معتدل و دسترسی به دریای خزر و جنگل‌های سرسبز، مواد اولیه تازه و متنوعی را در اختیار دارد که در آشپزی محلی به خوبی از آن‌ها استفاده می‌شود. در ادامه چند مورد از غذاهای معروف و محبوب استان گیلان را آمده است. باقلا قاتوق
------
طرز تهیه میرزا قاسمی: برای تهیه میرزا قاسمی، ابتدا بادمجا

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def compute_metrics(preds, refs, k, hit_at_ks, mrrs):
    """
    Compute metrics for predictions vs references.
    preds: list of predicted answers
    refs: list of ground truth answers
    hit_at_ks: list of per-query hit@k values
    mrrs: list of per-query reciprocal ranks
    """

    # --- Cosine Similarity (TF-IDF embedding) ---
    vectorizer = TfidfVectorizer().fit(refs + preds)
    ref_vecs = vectorizer.transform(refs)
    pred_vecs = vectorizer.transform(preds)
    cos_sims = [cosine_similarity(pred_vecs[i], ref_vecs[i])[0][0] for i in range(len(refs))]

    # --- Exact Match ---
    exact_matches = [1 if preds[i].strip() == refs[i].strip() else 0
                     for i in range(len(refs))]

    # --- Precision, Recall, F1 (token-based overlap) ---
    precision, recall, f1 = [], [], []
    for pred, ref in zip(preds, refs):
        pred_tokens, ref_tokens = set(pred.split()), set(ref.split())
        overlap = len(pred_tokens & ref_tokens)
        p = overlap / len(pred_tokens) if pred_tokens else 0
        r = overlap / len(ref_tokens) if ref_tokens else 0
        f = 2 * p * r / (p + r) if (p + r) > 0 else 0
        precision.append(p)
        recall.append(r)
        f1.append(f)

    metrics = {
        "cosine_similarity": np.mean(cos_sims),
        "MRR": np.mean(mrrs),
        "Precision": np.mean(precision),
        "Recall": np.mean(recall),
        "F1": np.mean(f1),
        "Exact_Match": np.mean(exact_matches),
        "Hit@k": np.mean(hit_at_ks)
    }
    return metrics



def evaluate_qa(test_data, full_qa, embedding_model_names, chunking_methods,
                k_values, alpha_values, qa_model, qa_tokenizer, max_new_tokens=256):

    queries = test_data['questions']
    real_answers = test_data['answers']

    results = []
    total_runs = len(embedding_model_names) * len(chunking_methods) * len(k_values) * len(alpha_values)
    run_count = 0

    for embedding_model_name in embedding_model_names:
        for chunking in chunking_methods:
            for k in k_values:
                for alpha in alpha_values:

                    run_count += 1
                    print(f"[{run_count}/{total_runs}] Running config → "
                          f"embedding={embedding_model_name}, chunking={chunking}, k={k}, alpha={alpha}")

                    preds, hit_at_ks, mrrs = [], [], []

                    for i, (query, ref) in enumerate(zip(queries, real_answers)):
                        try:
                            answer, contexts = full_qa(
                                query=query,
                                embedding_model_name=embedding_model_name,
                                chunking_type=chunking,
                                k=k,
                                alpha=alpha,
                                llm_model=qa_model,
                                llm_tokenizer=qa_tokenizer,
                                max_new_tokens=max_new_tokens
                            )
                        except Exception as e:
                            print(f"⚠️ Error on query {i}: {e}")
                            answer, contexts = "", []
                        preds.append(answer)

                        # --- Compute per-query Hit@k and MRR ---
                        hit, rr = 0, 0
                        for rank, ctx in enumerate(contexts):
                            if ref.lower() in ctx.lower():
                                hit = 1 if rank < k else 0
                                rr = 1.0 / (rank + 1)
                                break
                        hit_at_ks.append(hit)
                        mrrs.append(rr)

                    # --- Final metrics ---
                    metrics = compute_metrics(preds, real_answers, k, hit_at_ks, mrrs)
                    metrics.update({
                        "embedding_model": embedding_model_name,
                        "chunking": chunking,
                        "k": k,
                        "alpha": alpha
                    })
                    results.append(metrics)

                    print(f"✅ Finished config {run_count}/{total_runs} → "
                          f"Cosine={metrics['cosine_similarity']:.3f}, "
                          f"MRR={metrics['MRR']:.3f}, "
                          f"Hit@k={metrics['Hit@k']:.3f}, "
                          f"ExactMatch={metrics['Exact_Match']:.3f}, "
                          f"F1={metrics['F1']:.3f}, "
                          f"Precision={metrics['Precision']:.3f}, "
                          f"Recall={metrics['Recall']:.3f}")


    return pd.DataFrame(results)


In [ ]:
# Define configs
embedding_model_names = [
    "paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/distiluse-base-multilingual-cased-v2",
    "intfloat/multilingual-e5-base"
]

chunking_methods = ['meta', 'word', 'sentence', 'paragraph']
k_values = [2, 4, 6]
alpha_values = [0.2, 0.5, 0.8, 1.0]

results = evaluate_qa(
    test_data=test_data_for_rag,
    full_qa=full_qa,
    embedding_model_names=embedding_model_names,
    chunking_methods=chunking_methods,
    k_values=k_values,
    alpha_values=alpha_values,
    qa_model=llama_3_1b_model,
    qa_tokenizer=llama_3_1b_tokenizer,
    max_new_tokens=256
)

print(results)

[1/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=meta, k=2, alpha=0.2
✅ Finished config 1/144 → Cosine=0.285, MRR=0.184, Hit@k=0.211, ExactMatch=0.105, F1=0.272, Precision=0.310, Recall=0.255
[2/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=meta, k=2, alpha=0.5
✅ Finished config 2/144 → Cosine=0.648, MRR=0.579, Hit@k=0.632, ExactMatch=0.368, F1=0.602, Precision=0.673, Recall=0.584
[3/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=meta, k=2, alpha=0.8
✅ Finished config 3/144 → Cosine=0.629, MRR=0.500, Hit@k=0.579, ExactMatch=0.316, F1=0.545, Precision=0.597, Recall=0.555
[4/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=meta, k=2, alpha=1.0
✅ Finished config 4/144 → Cosine=0.617, MRR=0.421, Hit@k=0.474, ExactMatch=0.263, F1=0.511, Precision=0.598, Recall=0.528
[5/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=meta, k=4, al

Unsloth: Input IDs of shape torch.Size([1, 2139]) with length 2139 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


✅ Finished config 46/144 → Cosine=0.489, MRR=0.368, Hit@k=0.421, ExactMatch=0.211, F1=0.424, Precision=0.526, Recall=0.439
[47/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=paragraph, k=6, alpha=0.8
✅ Finished config 47/144 → Cosine=0.477, MRR=0.395, Hit@k=0.474, ExactMatch=0.105, F1=0.379, Precision=0.429, Recall=0.398
[48/144] Running config → embedding=paraphrase-multilingual-MiniLM-L12-v2, chunking=paragraph, k=6, alpha=1.0


Unsloth: Input IDs of shape torch.Size([1, 2168]) with length 2168 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


✅ Finished config 48/144 → Cosine=0.563, MRR=0.396, Hit@k=0.474, ExactMatch=0.263, F1=0.476, Precision=0.506, Recall=0.499
[49/144] Running config → embedding=sentence-transformers/distiluse-base-multilingual-cased-v2, chunking=meta, k=2, alpha=0.2
✅ Finished config 49/144 → Cosine=0.305, MRR=0.184, Hit@k=0.263, ExactMatch=0.105, F1=0.259, Precision=0.281, Recall=0.264
[50/144] Running config → embedding=sentence-transformers/distiluse-base-multilingual-cased-v2, chunking=meta, k=2, alpha=0.5
✅ Finished config 50/144 → Cosine=0.625, MRR=0.526, Hit@k=0.579, ExactMatch=0.368, F1=0.584, Precision=0.684, Recall=0.570
[51/144] Running config → embedding=sentence-transformers/distiluse-base-multilingual-cased-v2, chunking=meta, k=2, alpha=0.8
✅ Finished config 51/144 → Cosine=0.674, MRR=0.526, Hit@k=0.579, ExactMatch=0.368, F1=0.589, Precision=0.675, Recall=0.601
[52/144] Running config → embedding=sentence-transformers/distiluse-base-multilingual-cased-v2, chunking=meta, k=2, alpha=1.0
✅ Fi

In [ ]:
results

,cosine_similarity,MRR,Precision,Recall,F1,Exact_Match,Hit@k,embedding_model,chunking,k,alpha
0,0.285204,0.184211,0.310223,0.254674,0.271743,0.105263,0.210526,paraphrase-multilingual-MiniLM-L12-v2,meta,2,0.2
1,0.648386,0.578947,0.673148,0.584453,0.601584,0.368421,0.631579,paraphrase-multilingual-MiniLM-L12-v2,meta,2,0.5
2,0.628860,0.500000,0.596570,0.555416,0.545398,0.315789,0.578947,paraphrase-multilingual-MiniLM-L12-v2,meta,2,0.8
3,0.617218,0.421053,0.598401,0.527693,0.511193,0.263158,0.473684,paraphrase-multilingual-MiniLM-L12-v2,meta,2,1.0
4,0.430111,0.228070,0.449170,0.392563,0.398443,0.157895,0.368421,paraphrase-multilingual-MiniLM-L12-v2,meta,4,0.2
...,...,...,...,...,...,...,...,...,...,...,...
139,0.673211,0.385965,0.603954,0.594693,0.575617,0.315789,0.421053,intfloat/multilingual-e5-base,paragraph,4,1.0
140,0.372706,0.100000,0.417154,0.355443,0.352857,0.105263,0.263158,intfloat/multilingual-e5-base,paragraph,6,0.2
141,0.546034,0.368421,0.587086,0.507959,0.517624,0.263158,0.421053,intfloat/multilingual-e5-base,paragraph,6,0.5
142,0.557392,0.394737,0.515831,0.491651,0.484836,0.210526,0.473684,intfloat/multilingual-e5-base,paragraph,6,0.8


In [ ]:
results_sorted = pd.DataFrame(results).sort_values(
    by=[
        "cosine_similarity",
        "F1",
        "Precision",
        "Recall",
        "MRR",
        "Exact_Match",
        "Hit@k"
    ],
    ascending=False
).reset_index(drop=True)

# Reorder columns
results_sorted = results_sorted[
    [
        "cosine_similarity",
        "F1",
        "Precision",
        "Recall",
        "MRR",
        "Exact_Match",
        "Hit@k",
        "embedding_model",
        "chunking",
        "k",
        "alpha"
    ]
]

In [ ]:
results_sorted.head(100)

,cosine_similarity,F1,Precision,Recall,MRR,Exact_Match,Hit@k,embedding_model,chunking,k,alpha
0,0.763619,0.678015,0.742771,0.686360,0.456140,0.315789,0.578947,paraphrase-multilingual-MiniLM-L12-v2,meta,6,1.0
1,0.731440,0.649374,0.713733,0.658077,0.456140,0.315789,0.578947,intfloat/multilingual-e5-base,meta,6,1.0
2,0.713495,0.655148,0.722023,0.640451,0.539474,0.368421,0.631579,sentence-transformers/distiluse-base-multiling...,meta,4,0.5
3,0.706623,0.631232,0.693429,0.660045,0.456140,0.315789,0.578947,sentence-transformers/distiluse-base-multiling...,meta,6,1.0
4,0.689163,0.675155,0.800240,0.632282,0.500000,0.315789,0.578947,intfloat/multilingual-e5-base,meta,6,0.8
...,...,...,...,...,...,...,...,...,...,...,...
95,0.503490,0.504818,0.594175,0.477660,0.473684,0.210526,0.473684,paraphrase-multilingual-MiniLM-L12-v2,sentence,6,0.5
96,0.495993,0.468652,0.578420,0.438994,0.473684,0.210526,0.473684,intfloat/multilingual-e5-base,sentence,6,1.0
97,0.493131,0.472850,0.499614,0.482385,0.473684,0.263158,0.473684,paraphrase-multilingual-MiniLM-L12-v2,sentence,2,0.5
98,0.489014,0.423993,0.526078,0.439380,0.368421,0.210526,0.421053,paraphrase-multilingual-MiniLM-L12-v2,paragraph,6,0.5


In [56]:
for t, r in zip(test_data_for_rag['questions'], test_data_for_rag['answers']):
  answer, contexts = full_qa(t, "intfloat/multilingual-e5-base", 'meta',  llama_3_1b_model, llama_3_1b_tokenizer, k=6, alpha=1.0, max_new_tokens=256)
  print('answer:', answer)
  print('reference:', r)
  print('------')

answer: تخم ماهی (اشپل)، تخم مرغ و سبزیجات
reference: تخم ماهی (اشپل)، تخم مرغ و سبزیجات
------
answer: پنیر وورووشته، پنیر وروشون هم شناخته می شود
reference: پنیر وورووشته، پنیر وروشون
------
answer: شمار کشور
reference: شمال کشور
------
answer: نان تازه (به خصوص نان سنگک یا بربری) و همراه با سیر ترشی، زیتون پرورده و سبزی خوردن
reference: نان تازه.
------
answer: تنوع و طعم بی‌نظیر غذاهایش
reference: تنوع و طعم بی‌نظیر غذاهایش
------
answer: فسنجان
reference: فسنجان
------
answer: 2
reference: 2 پیمانه
------
answer: 4 عدد
reference: 4 عدد
------
answer: 20 تا 30 دقیقه
reference: حدود 20 تا 30 دقیقه
------
answer: طعم منحصربه‌فرد این کباب به دلیل استفاده از مواد اولیه‌ای چون گردو، رب انار و سبزیجات معطر محلی است که گوشت را طعم‌دار و ترد می‌کنند
reference: استفاده از مواد اولیه‌ای چون گردو، رب انار و سبزیجات معطر محلی
------
answer: 3-4
reference: 3-4 عدد
------
answer: روسی
reference: روسیه
------
answer: سیرواویج گیلانی
reference: سیراویج
------
answer: ۴ قاشق غذاخوری
reference: ۴ قا

# Step 10: Interface

In [57]:
!pip install gradio -q

import gradio as gr

# Wrapper function
def qa_interface(query):
    answer, contexts = full_qa(
        query,
        "intfloat/multilingual-e5-base",
        "meta",
        llama_3_1b_model,
        llama_3_1b_tokenizer,
        k=6,
        alpha=1.0,
        max_new_tokens=256
    )
    return answer

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🔍 Question Answering Demo")

    query = gr.Textbox(label="Enter your query", placeholder="Type your question here...")
    answer = gr.Textbox(label="Answer")

    query.submit(fn=qa_interface, inputs=query, outputs=answer)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fab236ca8fc09a5df1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
